Global setup

In [57]:
n=4
k=2
q=41 #Lithium AAA?  #same as Alkaline D
#q should be divisible by 8 for NTT
η=1 #secret key range, prefered 2 in Alkaline
γ=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2
R.<x> = ZZ[]

In [58]:
q

41

In [59]:
is_prime(q)

True

In [60]:
qlen = ceil(log(q,2))
qlen

6

In [61]:
τ = 1 #number of 1's in c
ln(binomial(4, τ)).n()+τ #challenge entropy, should be between n/2 and n

2.38629436111989

In [62]:
β = τ*η
β

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [63]:
((2*(γ-β)-1)/(2*γ-1))^(n*k).n()

0.586530272919658

Expected number of repetitions:

In [64]:
1/(((2*(γ-β)-1)/(2*γ-1))^(n*k))^2.n()

2.90682666192675

Alice generates a signing key

In [65]:
A = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
A

[ 8*x^3 + 20*x^2 + 13*x + 24 26*x^3 + 37*x^2 + 24*x + 12]
[ 30*x^3 + 13*x^2 + 3*x + 30   22*x^3 + 9*x^2 + 20*x + 9]

In [66]:
s1 = matrix([[randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)], 
            [randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)]])
s1

[     x^3 + x^2 + x]
[-x^3 - x^2 + x + 1]

In [67]:
s2 = matrix([[randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)], 
            [randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)]])
s2

[   x^2 + x + 1]
[-x^3 - x^2 - x]

$s_1$ and $s_2$ are private

In [68]:
t = matrix(((A*s1 + s2) % (x^n+1) % q))
t

[2*x^3 + 23*x^2 + 14*x + 7]
[ 6*x^3 + 3*x^2 + 5*x - 30]

$A$ and $t$ are public

Alice encodes her message

In [69]:
M = 'HI_BOB__'
M_list = [Mod(ord(ch) - ord("A") + 1,q).lift_centered() for ch in M]
M_list

[8, 9, -10, 2, 15, 2, -10, -10]

Pick nonces and generate $w_1$

In [70]:
y1 = matrix([[randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)], 
            [randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)]])
y1

[3*x^3 - 15*x^2 + 13*x + 7]
[ 15*x^3 - 5*x^2 - 2*x - 7]

In [71]:
y2 = matrix([[randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)], 
            [randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)]])
y2

[11*x^3 - 12*x^2 + 11*x - 6]
[   2*x^3 + 9*x^2 + 5*x + 6]

In [72]:
w = matrix(((A*y1+y2) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
w

[8*x^3 + 17*x^2 - 20*x - 11]
[2*x^3 - 18*x^2 - 12*x - 16]

Start the "non-cryptographic hash/XOF (combined?)"

In [73]:
w_list = [c for row in w for p in row for c in p.padded_list(n)]  #flatten the matrix down to coefficients
w_list

[-11, -20, 17, 8, -16, -12, -18, 2]

Diffusion box (D-box):

In [92]:
dboxlen = qlen

In [94]:
c_hash = format(vector(M_list).dot_product(vector(w_list)) % (2**dboxlen), f'0{dboxlen}b')
c_hash

'110010'

Use D-box output to sample a sparse polynomial

In [76]:
clenbits = log(n,2)
clenbits

2

In [77]:
def sample_in_ball(digest):  # digest should be at least ****
    c = [0] * n
    k = n
    for i in range(n-τ, n):
        while int(digest[k:k+clenbits], 2) > i:
            k = k+clenbits
        j = int(digest[k:k+clenbits], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+τ-n], 2)
        k = k+clenbits
    return c

Generate the signature


In [78]:
c = R(sample_in_ball(c_hash))
c

-x^2

In [79]:
z1 = matrix((y1 + c*s1) % (x^n+1))
z1

[2*x^3 - 15*x^2 + 14*x + 8]
[ 14*x^3 - 6*x^2 - 3*x - 8]

In [80]:
z2 = matrix((y2 + c*s2) % (x^n+1))
z2

[10*x^3 - 13*x^2 + 11*x - 5]
[   3*x^3 + 9*x^2 + 4*x + 5]

Is $\|\mathbf{z}_1\|_\infty$ or $\|\mathbf{z}_2
\|_\infty \geq \gamma-\beta$?

In [82]:
γ-β

15

If so, reject and pick new $\mathbf{y}_i$

If not, $(\mathbf{z}_1, \mathbf{z}_2, c)$ is the signature

In [83]:
(z1, z2, c)

(
[2*x^3 - 15*x^2 + 14*x + 8]  [10*x^3 - 13*x^2 + 11*x - 5]      
[ 14*x^3 - 6*x^2 - 3*x - 8], [   3*x^3 + 9*x^2 + 4*x + 5], -x^2
)

Bob verifies the signature

In [84]:
wprime = matrix((A*z1+z2-c*t) % (x^n+1) % q).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
wprime

[8*x^3 + 17*x^2 - 20*x - 11]
[2*x^3 - 18*x^2 - 12*x - 16]

Is $\|\mathbf{z}_i\|_\infty < \gamma-\beta$?

In [85]:
z1

[2*x^3 - 15*x^2 + 14*x + 8]
[ 14*x^3 - 6*x^2 - 3*x - 8]

In [86]:
z2

[10*x^3 - 13*x^2 + 11*x - 5]
[   3*x^3 + 9*x^2 + 4*x + 5]

In [87]:
γ-β

15

Yes

Is $c = H(M , w_1')$?

In [88]:
wprime_list = [c for row in wprime for p in row for c in p.padded_list(n)]  #flatten the matrix down to coefficients
wprime_list

[-11, -20, 17, 8, -16, -12, -18, 2]

In [89]:
cprime_hash = format(vector(M_list).dot_product(vector(w_list)) % (2**6), f'06b')
cprime_hash

'110010'

In [90]:
cprime = R(sample_in_ball(cprime_hash))
cprime

-x^2

In [91]:
c == cprime

True

Yes.  The signature is accepted.